In [1]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

In [2]:

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import os
# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path

from google.colab import drive 
drive.mount('/content/drive/')
!ls
%cd "drive"
%cd "MyDrive"
%cd "Ex4Data"
import import_ipynb
import utils


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
drive  sample_data
/content/drive
/content/drive/MyDrive
/content/drive/.shortcut-targets-by-id/1o0YBS756Yf3fr4s6rHsprRqjOfdY6PDU/Ex4Data
importing Jupyter notebook from utils.ipynb


In [3]:
###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################


# number of ResNet blocks for the first ResNet and the kernel size.
RESNET_1_BLOCKS = 3 
RESNET_1_KERNEL_SIZE = 15
RESNET_1_KERNEL_NUM = 64


###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################


# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 3
RESNET_2_KERNEL_SIZE = 3
RESNET_2_KERNEL_NUM = 32
DILATION = [1, 2, 4, 8]

# percentage of dropout for the dropout layer
DROPOUT = 0.15

# number of epochs, Learning rate and Batch size
EPOCHS = 60
LR = 0.01
BATCH = 32

In [4]:
def resnet_1(input_layer):  # TODO: implement this!
    """
    ResNet layer - input -> BatchNormalization -> Conv1D -> Relu -> BatchNormalization -> Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for _ in range(RESNET_1_BLOCKS):
      out_batchnorm = layers.BatchNormalization()(input_layer)
      out_conv1 = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same', activation='relu')(out_batchnorm)
      out_batchnorm = layers.BatchNormalization()(out_conv1)
      out_conv1 = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same', activation='relu')(out_batchnorm)
      input_layer = layers.Add()([input_layer, out_conv1])
    return input_layer





In [5]:
def resnet_2(input_layer):  # TODO: implement this!
    """
    Dilated ResNet layer - input -> BatchNormalization -> dilated Conv1D -> Relu -> BatchNormalization -> dilated Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for _ in range(RESNET_2_BLOCKS):
      for dilation in DILATION:
        out_batchnorm = layers.BatchNormalization()(input_layer)
        out_conv1 = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding='same', activation='relu', dilation_rate=dilation)(out_batchnorm)
        out_batchnorm = layers.BatchNormalization()(out_conv1)
        out_conv1 = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding='same', activation='relu', dilation_rate=dilation)(out_batchnorm)
        input_layer = layers.Add()([input_layer, out_conv1])
    return input_layer

In [6]:
def build_network():
    """
    builds the neural network architecture as shown in the exercise.
    :return: a Keras Model
    """
    ...
    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer)

    # # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding="same")(resnet_layer)

    # # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer)

    dropout_layer = layers.Dropout(DROPOUT)(resnet_layer)

    conv1d_layer = layers.Conv1D(np.ceil(RESNET_2_KERNEL_NUM/2), RESNET_2_KERNEL_SIZE, padding="same", activation='elu')(dropout_layer)

    output = layers.Dense(utils.OUTPUT_SIZE)(conv1d_layer)


    return tf.keras.Model(input_layer, output, name='from_seq_to_struct')



In [7]:
def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15,3))
    axes.plot(history.history['loss'][10:], label='Training loss')
    axes.plot(history.history['val_loss'][10:], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("/content/drive/MyDrive/Ex4Data/model_loss_history")  # TODO: you can change the path here


In [8]:
if __name__ == '__main__':
    model = build_network()

    # you can load here your input and output data
    X = np.load('input_data.npy')
    Y = np.load('label_data.npy')

    X_train, X_val, y_train, y_val  = train_test_split(X, Y, test_size=0.2, random_state=1)

    my_optimizer= tf.keras.optimizers.Adam(learning_rate=LR)
    model.compile(optimizer=my_optimizer, loss="mse")
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH, validation_data=(X_val, y_val))

    model.save("our_model.tf")
    model = tf.keras.models.load_model("our_model.tf")

    # part 3 predict
    to_predict = utils.generate_input(os.path.join("/content/drive/MyDrive/Ex4Data", "6xw6.pdb"))
    prediction = model.predict(to_predict[np.newaxis,:,:])
    matrix = utils.matrix_to_pdb(utils.get_seq_aa(os.path.join("/content/drive/MyDrive/Ex4Data", "6xw6.pdb"), "H")[0], prediction[0, :, :],"our_6xw6")


Epoch 1/60
50/50 [==============================] - 19s 73ms/step - loss: 32.4471 - val_loss: 15.9820
Epoch 2/60
50/50 [==============================] - 2s 35ms/step - loss: 4.3150 - val_loss: 14.1350
Epoch 3/60
50/50 [==============================] - 2s 35ms/step - loss: 3.1385 - val_loss: 5.9192
Epoch 4/60
50/50 [==============================] - 2s 35ms/step - loss: 2.6153 - val_loss: 3.6503
Epoch 5/60
50/50 [==============================] - 2s 34ms/step - loss: 2.2786 - val_loss: 2.7976
Epoch 6/60
50/50 [==============================] - 2s 34ms/step - loss: 2.1765 - val_loss: 2.4279
Epoch 7/60
50/50 [==============================] - 2s 34ms/step - loss: 1.9915 - val_loss: 1.8809
Epoch 8/60
50/50 [==============================] - 2s 34ms/step - loss: 1.7729 - val_loss: 1.9739
Epoch 9/60
50/50 [==============================] - 2s 34ms/step - loss: 1.7613 - val_loss: 1.6856
Epoch 10/60
50/50 [==============================] - 2s 35ms/step - loss: 1.5917 - val_loss: 1.2947
Epoch